In [1]:
#######################
# Interpolates some f #
#######################
import radiant as rad
import numpy as np


# Problem Parameters
a = -0.5
b = 0.5


def exact(*x):
    return np.prod(np.cos(np.asarray(x) * 2 * np.pi / (b - a)), axis=0)


def f(*x):
    return (len(x) * (2 * np.pi / (b - a)) ** 2 + 1) * exact(*x)  

In [2]:
# Parameters
d = 1
k = 1
inner = 5
outer = 3
start_delta = 1.
start_N = 3

# Computed Parameters
ranges = ((a, b),) * d
phi = rad.Wendland(d, k)
centre_thinning = 2. ** np.arange(0, inner)
delta_thinning = 2. ** np.arange(0, inner)
xcs = [rad.gridinc(ranges, inc, flat=True) for inc in (b - a) / (centre_thinning * start_N)]
deltas = start_delta / delta_thinning
integrator = rad.integrate.MeanIntegrator(ranges, 200)
# integrator = rad.integrate.QuadIntegrator(ranges)

# Solve for approximate solution
solver = rad.solve.MultilevelSolver(rad.solve.HelmholtzSolver, phi, deltas, xcs, integrator, outer=outer)
approx = solver.solve(f)
error = rad.error(exact, approx, integrator)

print("L2 Relative Error:", error)
print("Condition Number:", solver.cond())

AttributeError: 'MultilevelCompositeFunction' object has no attribute 'grad'

In [ ]:
rad.plot.spread(
    ranges,
    exact,
    approx,
    lambda *x: np.abs(exact(*x) - approx(*x)),
    n=200,
    wspace=0.5,
    hspace=0.5,
    titles=["Exact", "Approximate", "Abs. Error"],
    figsize=(15, 4)
)

In [ ]:
if d == 1:
    from IPython.display import HTML, display_html
    
    
    anim = rad.animate.ml_animate(ranges, exact, approx, n=200, figsize=(10, 4))
    display_html(HTML(anim.to_jshtml()))